# This notebook shows system identification of Franka Emika Panda

In [1]:
from mujoco_logger import SimLog

In [2]:
log = SimLog("data/panda_data.json")

In [3]:
# cut part of the data to disable the gripper
q = log.data("qpos")[:, :-2]
v = log.data("qvel")[:, :-2]
dv = log.data("qacc")[:, :-2]
tau = log.data("ctrl")[:, :-1]

q.shape, v.shape, dv.shape, tau.shape

((5000, 7), (5000, 7), (5000, 7), (5000, 7))

In [4]:
from mujoco import minimize

# minimize.least_squares(...)